In [99]:
import numpy as np
import pandas as pd
user_activity=pd.read_csv('Dataset/User_activity.txt')
device_demographics=pd.read_csv('Dataset/device_demographics.csv')
publishers=pd.read_csv('Dataset/publishers.csv')

user_activity_clean=user_activity.drop_duplicates().T
user_device_demographics_merge=user_activity_clean.merge(device_demographics,on='device_id')
user_device_publisher=user_device_demographics_merge.merge(publishers,on='app_name').drop_duplicates()
user_device_publisher=user_device_publisher[['device_id','gender_id','app_name','minutes','Publisher']].sort_values(['app_name','minutes'])

#### 5. Create an intermediate file (lookup file) using group by on column
# 'app_name' and
# 'Publisher' take the mode across gender id. E.g.

app_publisher_gender=user_device_publisher.groupby(['app_name','Publisher'])['gender_id'].agg(pd.Series.mode)
app_publisher_gender=pd.DataFrame(app_publisher_gender)

#### 6. For any NaN values in the gender_id column,
# replace missing values using the table created from step 5 ####
user_device_publisher.loc[user_device_publisher['gender_id'].isnull(),'gender_id']=user_device_publisher[user_device_publisher['gender_id'].isnull()].apply(lambda row:app_publisher_gender.loc[(row['app_name'],row['Publisher']),'gender_id'],axis=1)

#### 7. Create another intermediate file (lookup file) using group by on column 'app_name'
# and 'Publisher' take the median across minutes column id.

app_publisher_minutes=user_device_publisher.groupby(['app_name','Publisher'])['minutes'].agg(pd.Series.median)
app_publisher_minutes=pd.DataFrame(app_publisher_minutes)


#### 8. For any NaN values in the minute’s column, replace missing values using the table
# created from step 7
user_device_publisher.loc[user_device_publisher['minutes'].isnull(),'minutes']=user_device_publisher[user_device_publisher['minutes'].isnull()].apply(lambda row:app_publisher_minutes.loc[(row['app_name'],row['Publisher']),'minutes'],axis=1)

#### 9. Filter the data across minutes column if the value is
# greater than (median +
# standard deviation) across app_name or
# minutes value id less than 30 mins

minutes_filter=user_device_publisher.groupby('app_name')['minutes'].agg(['median','std'])
minutes_filter['sum']=minutes_filter['median']+minutes_filter['std']
# minutes_filter

convertdict = {'gender_id':int,'app_name':'string',
               'minutes':float,'Publisher':'string'
               }
user_device_publisher=user_device_publisher.astype(convertdict)
minutes_filter_2=user_device_publisher.apply(lambda row: True if (
            minutes_filter.loc[row['app_name'], 'sum'] >= row['minutes'] > 30) else False, axis=1)
user_device_publisher=user_device_publisher[minutes_filter_2]


#### 10. Aggregate the from output to step 9 and
# create a statistical dataset (as per 10.1 and
# 10.2) with count, min, mean, Qaurtile1 (25 percentile),
# median, Qaurtile2 (75
# percentile), standard deviation, and max values
# across App_Names, and across
# “app_name + gender_id”
# def percentile(n):
#     def percentile_(x):
#         return np.percentile(x, n)
#     percentile_.__name__ = 'percentile_%s' % n
#     return percentile_

#10.1
def Qaurtile1(x):
    return x.quantile(0.25)

def Qaurtile3(x):
    return x.quantile(0.75)


user_device_publisher_10=user_device_publisher.groupby('app_name')['minutes'].agg({'count','min','mean',Qaurtile1,'median',Qaurtile3,'std','max'})

#10.2


user_device_publisher_10_1= user_device_publisher.groupby(['app_name','gender_id'])['minutes'].agg({'count','min','mean','median','std','max'})

# Load the data created in step 10 into the target system. (export it to CSV file)
user_device_publisher_10.to_csv('step_10_csv')
user_device_publisher_10_1.to_csv('step_10_1_csv')

#### 11. Create a new data frame with columns as
# app_name, “total_minutes”, “total
#devices”, “Average Minutes per Device Per App”.
# Use the below expression for
#creating this column:

def my_agg(x):
    names = {
        'total_minutes': x['minutes'].sum(),
        'total_devices': x['device_id'].count(),
        'avg_time_spend_per_device':x['minutes'].mean()
    }

    return pd.Series(names)

user_device_publisher_new= user_device_publisher.groupby('app_name').apply(my_agg)

#### 12. Use data generated in step 11 to create a ranking system
# for Apps based on minutes spend over the app_name
# and rank based on total users on different apps and load
#the data into the target system
# (a separate file for each ranking system)

user_device_publisher_new['Rank_Duration_Based']=user_device_publisher_new['total_minutes'].rank()
user_device_publisher_new['Rank_User_Based']=user_device_publisher_new['total_devices'].rank(ascending=False)
sorted_data=user_device_publisher_new.sort_values("Rank_User_Based",ascending=False)

#### 13. Create and load files for Publisher Ranking as well

def total_devices_agg(x):
    names = {
        'total_devices': x['device_id'].count(),
    }

    return pd.Series(names)
user_device_publisher_13 = user_device_publisher.groupby('Publisher').apply(total_devices_agg)
# user_device_publisher['total_devices'].rank()
user_device_publisher_13['Rank']= user_device_publisher_13['total_devices'].rank(ascending=False)
user_device_publisher_13=user_device_publisher_13.sort_values('total_devices',ascending=False)

,total_devices,Rank
Publisher,,
Google Sites,5345,1.0
Facebook,4914,2.0
Amazon Sites,1083,3.0
"Snapchat, Inc",823,4.0
Oath,591,5.0
Pinterest,533,6.0
Netflix Inc.,433,7.0
Microsoft Sites,339,8.0
Wal-Mart,310,9.0
